In [70]:
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         pipeline)
import pandas as pd
import re
from itertools import product
from pprint import pprint
import numpy as np
import pylcs
import psycopg2 as p2
from psycopg2 import sql
from collections import Counter
from tqdm import tqdm
from rapidfuzz import fuzz

pd.set_option('display.width', 20000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)

In [71]:
dbname = "vulns_scanner"
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'

In [72]:
def get_df_from_bd(q):
    conn = p2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    cur.execute(q)
    colnames = [desc[0] for desc in cur.description]
    tuples = cur.fetchall()
    cur.close()
    df = pd.DataFrame(tuples, columns=colnames)
    return df

In [2]:
path_to_model = "/home/mikhail/Documents/pandan_study/vkr/vulns_scanner/mikhail_code/models/nuner2_v1_150325/best_model_tmp"
final_tokenizer = AutoTokenizer.from_pretrained(path_to_model, use_fast=True, add_prefix_space=True, local_files_only=True)
final_model = AutoModelForTokenClassification.from_pretrained(path_to_model, local_files_only=True)


In [4]:
df_test = pd.read_csv('df_100_not_in_stucco_v2.csv')

In [5]:
def extract_ners(cve, tokenizer=final_tokenizer, model=final_model):
    token_classifier = pipeline(
        "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
    )
    result = token_classifier(cve)
    vendor = []
    product = []
    version = []
    vendor_probs = []
    product_probs = []
    version_probs = []

    for ner_item in result:
        if ner_item['entity_group'] == 'vendor':
            vendor.append(str.lower(ner_item['word'].strip()))
            vendor_probs.append(str.lower(str(ner_item['score'])))
        elif ner_item['entity_group'] == 'product':
            product.append(str.lower(str(ner_item['word'].strip())))
            product_probs.append(str.lower(str(ner_item['score'])))
        elif ner_item['entity_group'] == 'version':
            version.append(str.lower(str(ner_item['word'].strip())))
            version_probs.append(str.lower(str(ner_item['score'])))
    return {'ners': [vendor, product, version], 'scores': [vendor_probs, product_probs, version_probs]}

In [6]:
df_test[['ners_list', 'scores_list']] = df_test['descr'].apply(lambda x: extract_ners(x)).apply(pd.Series)
df_test['vendor_ner'], df_test['product_ner'], df_test['version_ner'] = zip(*df_test['ners_list'])
df_test['vendor_score_ner'], df_test['product_score_ner'], df_test['version_score_ner'] = zip(*df_test['scores_list'])

Device set to use cpu
/home/mikhail/.local/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set t

In [7]:
def deduplicate_using_probs(row, ner_type):
    if ner_type == 'product':
        product_ner = row['product_ner']
        product_score_ner = row['product_score_ner']
        if not product_ner or len(product_ner) == 1:
            return product_ner, product_score_ner
        else:
            max_idx = product_score_ner.index(max(product_score_ner))
            return [[product_ner[max_idx]], [product_score_ner[max_idx]]]
    elif ner_type == 'vendor':
        vendor_ner = row['vendor_ner']
        vendor_score_ner = row['vendor_score_ner']
        if not vendor_ner or len(vendor_ner) == 1:
            return vendor_ner, vendor_score_ner
        else:
            max_idx = vendor_score_ner.index(max(vendor_score_ner))
            return [[vendor_ner[max_idx]], [vendor_score_ner[max_idx]]]



In [8]:
df_test[['dedup_vendor', 'dedup_vendor_score']] = df_test.apply(lambda x: deduplicate_using_probs(x, 'vendor'), axis=1).apply(pd.Series)
df_test[['dedup_product', 'dedup_product_score']] = df_test.apply(lambda x: deduplicate_using_probs(x, 'product'), axis=1).apply(pd.Series)

In [21]:
df_test

,cve_id,cpe_id_pk,vendor,product,version,descr,initial_cpe,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score
0,CVE-2023-0878,294697,nuxt,nuxt,1.0.0,Cross-site Scripting (XSS) - Generic in GitHub...,cpe:2.3:a:nuxt:nuxt:1.0.0:rc8:*:*:*:*:*:*,"[[], [github repository], [to 3.2.1.]]","[[], [0.88403636], [0.799739]]",[],[github repository],[to 3.2.1.],[],[0.88403636],[0.799739],[],[],[github repository],[0.88403636]
1,CVE-2008-0444,273643,elog,elog,2.2.0,Cross-site scripting (XSS) vulnerability in El...,cpe:2.3:a:elog:elog:2.2.0:*:*:*:*:*:*:*,"[[electronic], [], [before 2.7.0]]","[[0.46619815], [], [0.99990386]]",[electronic],[],[before 2.7.0],[0.46619815],[],[0.99990386],[electronic],[0.46619815],[],[]
2,CVE-2020-14518,460900,philips,dreammapper,2.17.1,"Philips DreamMapper, Version 2.24 and prior. I...",cpe:2.3:a:philips:dreammapper:2.17.1:*:*:*:*:i...,"[[philips], [dreammapper,], [2.24]]","[[0.9992367], [0.9967809], [0.9996884]]",[philips],"[dreammapper,]",[2.24],[0.9992367],[0.9967809],[0.9996884],[philips],[0.9992367],"[dreammapper,]",[0.9967809]
3,CVE-2022-27858,141525,activity_log_project,activity_log,2.2.8,CSV Injection vulnerability in Activity Log Te...,cpe:2.3:a:activity_log_project:activity_log:2....,"[[], [activity log team activity log], [2.8.3]]","[[], [0.99662894], [0.99979264]]",[],[activity log team activity log],[2.8.3],[],[0.99662894],[0.99979264],[],[],[activity log team activity log],[0.99662894]
4,CVE-2022-4725,214060,amazon,aws_software_development_kit,2.2.4,A vulnerability was found in AWS SDK 2.59.0. I...,cpe:2.3:a:amazon:aws_software_development_kit:...,"[[], [aws sdk], [2.59.0., 2.59.1]]","[[], [0.9994598], [0.99985254, 0.9996922]]",[],[aws sdk],"[2.59.0., 2.59.1]",[],[0.9994598],"[0.99985254, 0.9996922]",[],[],[aws sdk],[0.9994598]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CVE-2023-4827,389896,ninjateam,filester,1.5.1,The File Manager Pro WordPress plugin before 1...,cpe:2.3:a:ninjateam:filester:1.5.1:*:*:*:*:wor...,"[[], [file manager pro, wordpress], [before 1.8]]","[[], [0.90689164, 0.73657227], [0.9998642]]",[],"[file manager pro, wordpress]",[before 1.8],[],"[0.90689164, 0.73657227]",[0.9998642],[],[],[file manager pro],[0.90689164]
96,CVE-2021-34628,629697,weblizar,admin_custom_login,2.4.2,The Admin Custom Login WordPress plugin is vul...,cpe:2.3:a:weblizar:admin_custom_login:2.4.2:*:...,"[[], [login, wordpress], [3.2.7.]]","[[], [0.95257753, 0.5439362], [0.99975437]]",[],"[login, wordpress]",[3.2.7.],[],"[0.95257753, 0.5439362]",[0.99975437],[],[],[login],[0.95257753]
97,CVE-2017-16083,665237,node-simple-router,node-simple-router,0.9.1-3,node-simple-router is a minimalistic router fo...,cpe:2.3:a:node-simple-router:node-simple-route...,"[[], [node-simple-router], []]","[[], [0.9974068], []]",[],[node-simple-router],[],[],[0.9974068],[],[],[],[node-simple-router],[0.9974068]
98,CVE-2017-5869,673773,nuxeo,nuxeo,6.0,Directory traversal vulnerability in the file ...,cpe:2.3:a:nuxeo:nuxeo:6.0:hotfix38:*:*:*:*:*:*,"[[], [nuxeo platform], [6.0,, 7.1,, 7.2,, 7.3]]","[[], [0.8639369], [0.9998055, 0.99982196, 0.99...",[],[nuxeo platform],"[6.0,, 7.1,, 7.2,, 7.3]",[],[0.8639369],"[0.9998055, 0.99982196, 0.99956363, 0.9997532]",[],[],[nuxeo platform],[0.8639369]


In [9]:
def extract_version(matched):
        if matched:
            version = matched.group('version')
            # Normalize separators (replace '-' with '.' if needed)
            version = version.replace('-', '.')
            return version
        return None

def classify_version_string(version_str):
    """
    Generate all versions for expressions containing 'before'.
    Handles two cases:
    1) "before X.Y.Z" - generates all versions up to X.Y.Z
    2) "A.B.x before A.B.C" - generates all patch versions A.B.0 to A.B.(C-1)
    """
    version_str = str.lower(version_str)



    # through, including
    group_name = 'through'
    group_words = ['through', 'earlier', '<=', 'prior', 'up to', 'up to, and including', 'up to and including', 'older']
    for group_wrd in group_words:
        if group_wrd in version_str:
            # More complicated multi version logic
            # '2.1 through 3.17'
            multi_version_pattern = r'(?P<version1>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)\s*' \
                        r'(?:through|earlier|prior|\<\=|up to)\s*' \
                        r'(?P<version2>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)'
            multi_match = re.search(multi_version_pattern, version_str, re.IGNORECASE)
            if multi_match:
                version1 = multi_match.group('version1')
                version2 = multi_match.group('version2')
                return [version1, version2], f'{group_name} multi-match'

            pattern = (
                r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)'  # Version with digits/x and separators
            )
            matched = re.search(pattern, version_str, re.IGNORECASE)
            return [extract_version(matched)], f'{group_name} group'

    # before, not including
    group_name = 'before'
    group_words = ['before', '<']
    for group_wrd in group_words:
        if group_wrd in version_str:
            # More complicated multi version logic
            # '4.2.x before 4.2.8'
            multi_version_pattern = r'(?P<version1>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)\s*' \
                        r'(?:before)\s*' \
                        r'(?P<version2>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)'
            multi_match = re.search(multi_version_pattern, version_str, re.IGNORECASE)
            if multi_match:
                version1 = multi_match.group('version1')
                version2 = multi_match.group('version2')
                return [version1, version2], f'{group_name} multi-match'

            # if simple logic
            pattern = (
                r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)'  # Version with digits/x and separators
            )
            matched = re.search(pattern, version_str, re.IGNORECASE)
            return [extract_version(matched)], f'{group_name} group'

    # after, including
    group_name = 'after'
    group_words = ['after', '>=']
    for group_wrd in group_words:
        if group_wrd in version_str:
            # More complicated multi version logic
            # '4.2.x before 4.2.8'
            multi_version_pattern = r'(?P<version1>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)\s*' \
                        r'(?:older|after|\>\=)\s*' \
                        r'(?P<version2>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)'
            multi_match = re.search(multi_version_pattern, version_str, re.IGNORECASE)
            if multi_match:
                version1 = multi_match.group('version1')
                version2 = multi_match.group('version2')
                return [version1, version2], f'{group_name} multi-match'

            # if simple logic
            pattern = (
                r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)'  # Version with digits/x and separators
            )
            matched = re.search(pattern, version_str, re.IGNORECASE)
            return [extract_version(matched)], f'{group_name} group'

    # between
    group_name = 'between'
    group_words = ['between', 'to', ' - ']
    for group_wrd in group_words:
        if group_wrd in version_str:
            # More complicated multi version logic
            # '4.2.x before 4.2.8'
            multi_version_pattern = r'(?P<version1>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)\s*' \
                        r'(?:between|to)\s*' \
                        r'(?P<version2>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)'
            multi_match = re.search(multi_version_pattern, version_str, re.IGNORECASE)
            if multi_match:
                version1 = multi_match.group('version1')
                version2 = multi_match.group('version2')
                return [version1, version2], f'{group_name} multi-match'

            # # if simple logic
            # pattern = (
            #     r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)\s*)'  # Version with digits/x and separators
            # )
            # matched = re.search(pattern, version_str, re.IGNORECASE)
            # return extract_version(matched), f'{group_name} group'



    pattern = (
        r'(?:v|version)?\s*'  # Optional 'v' or 'version'
        r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)'  # Version with digits/x and separators
    )
    matched = re.search(pattern, version_str, re.IGNORECASE)
    return [extract_version(matched)], 'other'

In [12]:
def parse_version(version_str):
    components = re.findall(r'\d+|x', version_str, re.IGNORECASE)
    parsed = []
    for c in components:
        if c.lower() == 'x':
            parsed.append('x')
        else:
            parsed.append(int(c))
    return parsed

In [13]:
def generate_versions(versions, group_name, debug=False):
    if group_name == 'other':
        result = [versions[0]]
        version_other = parse_version(versions[0])
        while len(version_other) != 3:
            if len(version_other) > 3:
                result.append('.'.join([str(x) for x in version_other]))
                version_other.pop()
            elif len(version_other) < 3:
                result.append('.'.join([str(x) for x in version_other]))
                version_other.append(0)
        else:
            result.append('.'.join([str(x) for x in version_other]))
        # print(f'result: {result}')
        # print(f'other versions: {other_versions}')
        # result_merged = result + other_versions
        # print(f'joined: {result_merged}')
        return result
        # return versions
    group_type = group_name.split()[0].lower()

    if len(versions) == 1:
        if group_type == 'before':
            return generate_versions(['0.0.0', versions[0]], 'before multi-match', debug=debug)
        elif group_type == 'through':
            return generate_versions(['0.0.0', versions[0]], 'through multi-match', debug=debug)
        # here access DB and query max version?
        elif group_type == 'after':
            return generate_versions([versions[0], '20.0.0'], 'after multi-match', debug=debug)
        else:
            return []
    elif len(versions) >= 1:
        # for ['3.x', '3.1.1']
        # 3.x
        start = parse_version(versions[0])
        len_original_start = len(start)
        # 3.1.1
        end = parse_version(versions[1])
        len_original_end = len(end)
        # normalize versions
        while len(start) != 3:
            if len(start) > 3:
                start.pop()
            elif len(start) < 3:
                start.append(0)

        while len(end) != 3:
            if len(end) > 3:
                end.pop()
            elif len(end) < 3:
                end.append(0)

        possible_values = []
        if debug:
            print(f'start version: {start}, end version: {end}')
            print(f'len_original_end: {len_original_end}')
        for i in range(3):
            # print(f'possible values: {possible_values}')
            # 3
            start_comp = start[i]
            # 3
            end_comp = end[i]
            if debug:
                print(f'Start component: {start_comp}, End component: {end_comp}')

            if start_comp == 'x':
                # Надо как-то проверять, нужно ли генерировать такик большие числа версий
                if 'before' in group_type:
                    max_val = end_comp - 1 if isinstance(end_comp, int) else 99
                else:
                    max_val = end_comp if isinstance(end_comp, int) else 99
                possible_values.append(list(range(0, max_val + 1)))

                continue
            if isinstance(start_comp, int):
                if isinstance(end_comp, str) and end_comp.lower() == 'x':
                    end_comp = 99  # High maximum for 'x' in end
                if start_comp > end_comp:
                    return []
                if start_comp < end_comp:
                    if 'before' in group_type:
                        current_max = end_comp - 1
                    else:
                        current_max = end_comp
                    # possible_values.append(list(range(start_comp, current_max + 1)))
                    possible_values.append(list(range(start_comp, 10)))

                    # Allow any values for remaining components
                    for j in range(i + 1, 3):
                        possible_values.append(list(range(0, 100)))  # Arbitrary high limit
                    break
                else:
                    possible_values.append([start_comp])
            else:
                # print(f'possible values: {possible_values}')
                possible_values.append([0])


        if debug:
            print(f'possible values: {possible_values}')
        if 'x' not in end and 'x' not in start:
            generated_components = list(product(*possible_values))
            if debug:
                print(f'generated components: {generated_components[:10]}')
            generated_components_to_use = []
            for val in generated_components:
                if not (((val[0] == end[0]
                        and val[1] > end[1]) or
                        (val[0] == end[0]
                        and val[1] == end[1]
                        and val[2] > end[2]) or
                        val[0] > end[0])
                    or ((val[0] == start[0]
                         and val[1] < start[1]) or
                        (val[0] == start[0]
                         and val[1] == start[1]
                         and val[2] < start[2])) or
                        val[0] < start[0]):
                    generated_components_to_use.append(val)
            if debug:
                print(f'generated components to use: {generated_components_to_use[:10], generated_components_to_use[-10:]}')
            versions_list = ['.'.join(map(str, v)) for v in generated_components_to_use]

            # return versions_list
        else:
            generated_components = list(product(*possible_values))
            versions_list = ['.'.join(map(str, v)) for v in generated_components]
        if len_original_end == 2 or len_original_start == 2:
            versions_set = []
            for x in versions_list:
                versions_set.append(x.split('.')[:2])
            versions_set = set(['.'.join(y) for y in versions_set])
            # for x in versions_list:
            versions_list.extend(list(versions_set))
        if debug:
            print(versions_list[:10])
        return versions_list
    else:
        # print('last else')
        return []

In [ ]:
r = []
d_vers = {}
cve_to_vers = {}
d = {}
for i, row in df_test.iterrows():
    possible_versions = []
    # print(i, row['version_ner'])
    for version_ner in row['version_ner']:
        # print(version_ner)
        preprocessed_ner = classify_version_string(version_ner)
        if preprocessed_ner[0][0] is None:
            continue
        # print(preprocessed_ner, end='\n'+'*'*50+'\n')
        generated_versions = generate_versions(*preprocessed_ner)
        possible_versions.extend(generated_versions)
    r.append(1 if row['version'] in possible_versions else 0)
    d_vers[(row['version'], row['cve_id'])] = 
    cve_to_vers[row['cve_id']] = row['version']
    d[row['cve_id']] = possible_versions

In [98]:
df_test['true_version_in_predicted'] = r

In [26]:
sum(r)

50

In [27]:
df_test.sample()

,cve_id,cpe_id_pk,vendor,product,version,descr,initial_cpe,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score
52,CVE-2023-49799,395067,johannschopplich,nuxt_api_party,0.16.1,`nuxt-api-party` is an open source module to p...,cpe:2.3:a:johannschopplich:nuxt_api_party:0.16...,"[[], [], [0.22.1.]]","[[], [], [0.9995301]]",[],[],[0.22.1.],[],[],[0.9995301],[],[],[],[]


In [68]:
count_vendor = 0
count_product = 0
for i, row in df_test.iterrows():

    if row['dedup_vendor']:
        if (row['vendor'] == row['dedup_vendor'][0]
            or row['vendor'] == '_'.join(row['dedup_vendor'][0].split())):
            count_vendor += 1
    if row['dedup_product']:
        if (row['product'] == row['dedup_product'][0]
            or row['product'] == '_'.join(row['dedup_product'][0].split())):
            count_product += 1    
        else:
            print(row['product'], row['dedup_product'])

nuxt ['github repository']
dreammapper ['dreammapper,']
activity_log ['activity log team activity log']
aws_software_development_kit ['aws sdk']
data-connector-rock ['apollos apps']
cdh ['cloudera cdh']
advisor_network ['advisor network)']
mobile_security_framework ['security']
sights_n_sounds_streaming_media_server ['media server']
wp_matterport_shortcode ['wp matterport shortcode plugin']
ovirt-engine ['ovirt']
jasperreports_library ['jasperreports']
product_slider_for_woocommerce ['wordpress']
etsy_shop ['sheedy etsy shop']
pfsense ['pfsense plus']
k-9_mail ['k9mail']
integration_bus ['websphere message broker']
dashboard_widget_suite ['dashboard widgets suite']
handlebars.js ['handlebars']
octopus_server ['octopus deploy']
vk_all_in_one_expansion_unit ['wordpress']
print_invoice_\&_delivery_notes_for_woocommerce ['woocommerce']
automatewoo ['woocommerce']
jdk ['java se']
discy ['wordpress']
icewall_sso_agent ['icewall file manager']
private_instant_verified_transactions ['anonymous

In [64]:
row['dedup_vendor']

['ibm']

Это те, что сразу совпали. При этом часть названий продуктов, похожи, но не точно. Поэтому нужно использовать поиск по строкам. 

In [67]:
print(count_vendor, count_product)

21 34


In [25]:
cve_to_vers

{'CVE-2023-0878': '1.0.0', 'CVE-2008-0444': '2.2.0', 'CVE-2020-14518': '2.17.1', 'CVE-2022-27858': '2.2.8', 'CVE-2022-4725': '2.2.4', 'CVE-2021-32691': '1.2.5', 'CVE-2016-4572': '5.5.1', 'CVE-2023-31441': '2.2.0', 'CVE-2023-42261': '0.9.4.1', 'CVE-2022-36532': '3.0.2', 'CVE-2023-5654': '4.10.4', 'CVE-2005-4194': nan, 'CVE-2023-35094': '1.7.1', 'CVE-2014-7851': '3.3', 'CVE-2018-5429': '6.3.2', 'CVE-2004-0431': '6.5', 'CVE-2023-0166': '1.12.16', 'CVE-2017-12976': '6.20160211', 'CVE-2021-21423': '0.15.3', 'CVE-2021-3765': '3.19.1', 'CVE-2019-11504': '0.24.1', 'CVE-2023-25975': '0.11', 'CVE-2022-23993': '2.3.0', 'CVE-2018-1000831': '5.200', 'CVE-2017-1207': '9.0.0', 'CVE-2017-1723': '7.2.8', 'CVE-2015-8360': '2.5.5', 'CVE-2015-3933': '0.0.3', 'CVE-2015-1913': '8.0.1.4', 'CVE-2022-36296': '0.1.64', 'CVE-2021-43009': '9.9', 'CVE-2023-26517': '1.5', 'CVE-2019-19919': '4.0.3', 'CVE-2017-11348': '3.2.20', 'CVE-2022-24124': '1.7.2', 'CVE-2023-1554': '5.7.4', 'CVE-2023-0230': '3.4.0', 'CVE-2016-9

In [39]:
# pprint({x:y for x,y in d_vers.items() if y == 0})

In [69]:
def get_lcs(ner_name, unique_entities):
    lcs_scores = np.array(pylcs.lcs_of_list(ner_name, unique_entities))
    if len(np.argwhere(lcs_scores == np.max(lcs_scores))) <= 1:
        return unique_entities[np.argmax(lcs_scores)], np.max(lcs_scores)
    else:
        candidates = unique_entities[np.argwhere(lcs_scores == np.max(lcs_scores))]
        len_of_query = len(ner_name)
        d = -1
        fit_cand = ''
        for cand in candidates:
            cand = cand[0]
            diff = abs(len_of_query - len(cand))
            # print(cand, d, diff)
            if d == -1:
                d = diff
                fit_cand = cand
            elif d > diff:
                d = diff
                fit_cand = cand
            else:
                continue
        return fit_cand, np.max(lcs_scores)


In [75]:
product_ner = df_test['dedup_product'].astype(str).apply(lambda x: x.lstrip('[\'').rstrip(']\'')).values
vendor_ner = df_test['dedup_vendor'].astype(str).apply(lambda x: x.lstrip('[\'').rstrip(']\'')).values

In [76]:
df_all = get_df_from_bd('select * from cpes limit 1000000;')

In [78]:
unique_products = df_all['product'].unique()
unique_vendors = df_all['vendor'].unique()

In [84]:
matched_db_product = []
matched_db_vendor = []
for pr in product_ner:
    if pr:
        print(f'Product NER: {pr}')
        # (prod, score)= get_lcs_nonnorm(pr, unique_products)
        # print('Old')
        # print(f'Found product in DB: {prod}')
        # print(f'score: {score}')
        # print('Normed')
        (prod, score)= get_lcs(pr, unique_products)
        print(f'Found product in DB: {prod}')
        print(f'score: {score}', end='\n\n')
        df_all = get_df_from_bd(f"select * from cpes where product = '{prod}' limit 1;")
        matched_db_vendor.append(df_all['vendor'].values[0])
        matched_db_product.append(df_all['product'].values[0])
    else:
        matched_db_product.append('')
        matched_db_vendor.append('')

Product NER: github repository
Found product in DB: centricity_clinical_archive_audit_trail_repository
score: 14

Product NER: dreammapper,
Found product in DB: dreammapper
score: 11

Product NER: activity log team activity log
Found product in DB: the_university_of_cambridge_web_authentication_system_apache_authentication_agent
score: 19

Product NER: aws sdk
Found product in DB: drawings_sdk
score: 6

Product NER: apollos apps
Found product in DB: woocommerce_pdf_invoices\,_packing_slips\,_delivery_notes_and_shipping_labels
score: 10

Product NER: cloudera cdh
Found product in DB: cloudera_cdh
score: 11

Product NER: advisor network)
Found product in DB: advisor_network
score: 14

Product NER: security
Found product in DB: security
score: 8

Product NER: bolt cms
Found product in DB: bolt_cms
score: 7

Product NER: media server
Found product in DB: media_server
score: 11

Product NER: wp matterport shortcode plugin
Found product in DB: wp_matterport_shortcode
score: 21

Product NER: 

In [85]:
df_test['matched_db_product'] = matched_db_product
df_test['matched_db_vendor'] = matched_db_vendor

In [87]:
df_test.sample(2)

,cve_id,cpe_id_pk,vendor,product,version,descr,initial_cpe,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score,matched_db_product,matched_db_vendor
52,CVE-2023-49799,395067,johannschopplich,nuxt_api_party,0.16.1,`nuxt-api-party` is an open source module to proxy API requests. nuxt-api-party attempts to check if the user has passed an absolute URL to prevent the aforementioned attack. This has been recentl...,cpe:2.3:a:johannschopplich:nuxt_api_party:0.16.1:*:*:*:*:node.js:*:*,"[[], [], [0.22.1.]]","[[], [], [0.9995301]]",[],[],[0.22.1.],[],[],[0.9995301],[],[],[],[],,
29,CVE-2022-36296,174494,jumpdemand,activedemand,0.1.64,Broken Authentication vulnerability in JumpDEMAND Inc. ActiveDEMAND plugin <= 0.2.27 at WordPress allows unauthenticated post update/create/delete.\n\n,cpe:2.3:a:jumpdemand:activedemand:0.1.64:*:*:*:*:wordpress:*:*,"[[], [jumpdemand, activedemand], [inc., 0.2.27]]","[[], [0.9975235, 0.9986274], [0.7742043, 0.9997603]]",[],"[jumpdemand, activedemand]","[inc., 0.2.27]",[],"[0.9975235, 0.9986274]","[0.7742043, 0.9997603]",[],[],[activedemand],[0.9986274],activedemand,jumpdemand


In [90]:
df_test['product'].iloc[1]

'elog'

In [91]:
df_test[df_test['matched_db_product'] == df_test['product']].shape


(40, 21)

In [97]:
matched_db_product

['centricity_clinical_archive_audit_trail_repository', '', 'dreammapper', 'the_university_of_cambridge_web_authentication_system_apache_authentication_agent', 'drawings_sdk', 'woocommerce_pdf_invoices\\,_packing_slips\\,_delivery_notes_and_shipping_labels', 'cloudera_cdh', 'advisor_network', 'security', 'bolt_cms', '', 'media_server', 'wp_matterport_shortcode', 'ovirt', 'jasperreports', 'quicktime', 'wordpress', 'git-annex', '', '', 'zotonic', 'user_management_system_in_php_stored_procedure', 'pfsense_plus', 'k-9_mail', 'websphere_message_broker', '', 'bamboo', 'genixcms', 'rational_test_workbench', 'activedemand', 'opmon', 'dashboard_widget_suite', 'handlebars', 'octopus_deploy', 'casdoor', 'quick_paypal_payments', 'wordpress', 'botan', 'rsvpmaker', '', 'sandstorm', 'woocommerce', 'node-opencv', 'woocommerce', '', '', 'java_se', 'wordpress', 'commons_fileupload', 'icewall_file_manager', 'whale', 'mailbird', '', 'anonymous_post_pro', 'junos_space', 'polarssl', 'xlockmore', 'performance

In [96]:
df_test.head()

,cve_id,cpe_id_pk,vendor,product,version,descr,initial_cpe,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score,matched_db_product,matched_db_vendor
0,CVE-2023-0878,294697,nuxt,nuxt,1.0.0,Cross-site Scripting (XSS) - Generic in GitHub repository nuxt/framework prior to 3.2.1.,cpe:2.3:a:nuxt:nuxt:1.0.0:rc8:*:*:*:*:*:*,"[[], [github repository], [to 3.2.1.]]","[[], [0.88403636], [0.799739]]",[],[github repository],[to 3.2.1.],[],[0.88403636],[0.799739],[],[],[github repository],[0.88403636],centricity_clinical_archive_audit_trail_repository,gehealthcare
1,CVE-2008-0444,273643,elog,elog,2.2.0,Cross-site scripting (XSS) vulnerability in Electronic Logbook (ELOG) before 2.7.0 allows remote attackers to inject arbitrary web script or HTML via subtext parameter to unspecified components.,cpe:2.3:a:elog:elog:2.2.0:*:*:*:*:*:*:*,"[[electronic], [], [before 2.7.0]]","[[0.46619815], [], [0.99990386]]",[electronic],[],[before 2.7.0],[0.46619815],[],[0.99990386],[electronic],[0.46619815],[],[],,
2,CVE-2020-14518,460900,philips,dreammapper,2.17.1,"Philips DreamMapper, Version 2.24 and prior. Information written to log files can give guidance to a potential attacker.",cpe:2.3:a:philips:dreammapper:2.17.1:*:*:*:*:iphone_os:*:*,"[[philips], [dreammapper,], [2.24]]","[[0.9992367], [0.9967809], [0.9996884]]",[philips],"[dreammapper,]",[2.24],[0.9992367],[0.9967809],[0.9996884],[philips],[0.9992367],"[dreammapper,]",[0.9967809],dreammapper,philips
3,CVE-2022-27858,141525,activity_log_project,activity_log,2.2.8,CSV Injection vulnerability in Activity Log Team Activity Log <= 2.8.3 on WordPress.\n\n,cpe:2.3:a:activity_log_project:activity_log:2.2.8:*:*:*:*:wordpress:*:*,"[[], [activity log team activity log], [2.8.3]]","[[], [0.99662894], [0.99979264]]",[],[activity log team activity log],[2.8.3],[],[0.99662894],[0.99979264],[],[],[activity log team activity log],[0.99662894],the_university_of_cambridge_web_authentication_system_apache_authentication_agent,cam
4,CVE-2022-4725,214060,amazon,aws_software_development_kit,2.2.4,A vulnerability was found in AWS SDK 2.59.0. It has been rated as critical. This issue affects the function XpathUtils of the file aws-android-sdk-core/src/main/java/com/amazonaws/util/XpathUtils....,cpe:2.3:a:amazon:aws_software_development_kit:2.2.4:*:*:*:*:android:*:*,"[[], [aws sdk], [2.59.0., 2.59.1]]","[[], [0.9994598], [0.99985254, 0.9996922]]",[],[aws sdk],"[2.59.0., 2.59.1]",[],[0.9994598],"[0.99985254, 0.9996922]",[],[],[aws sdk],[0.9994598],drawings_sdk,opendesign


In [ ]:
df_test[df_test['matched_db_product'] != df_test['product']][['cve_id', 'product', 'matched_db_product']]

,cve_id,product,matched_db_product
0,CVE-2023-0878,nuxt,centricity_clinical_archive_audit_trail_repository
1,CVE-2008-0444,elog,
3,CVE-2022-27858,activity_log,the_university_of_cambridge_web_authentication_system_apache_authentication_agent
4,CVE-2022-4725,aws_software_development_kit,drawings_sdk
5,CVE-2021-32691,data-connector-rock,"woocommerce_pdf_invoices\,_packing_slips\,_delivery_notes_and_shipping_labels"
6,CVE-2016-4572,cdh,cloudera_cdh
8,CVE-2023-42261,mobile_security_framework,security
10,CVE-2023-5654,react-devtools,
11,CVE-2005-4194,sights_n_sounds_streaming_media_server,media_server
13,CVE-2014-7851,ovirt-engine,ovirt


In [95]:
df_test[df_test['matched_db_vendor'] != df_test['vendor']][['cve_id', 'vendor', 'matched_db_vendor']]


,cve_id,vendor,matched_db_vendor
0,CVE-2023-0878,nuxt,gehealthcare
1,CVE-2008-0444,elog,
3,CVE-2022-27858,activity_log_project,cam
4,CVE-2022-4725,amazon,opendesign
5,CVE-2021-32691,apollosapp,webtoffee
8,CVE-2023-42261,opensecurity,eset
10,CVE-2023-5654,facebook,
11,CVE-2005-4194,innovateware,plex
13,CVE-2014-7851,redhat,ovirt
14,CVE-2018-5429,tibco,jaspersoft


In [102]:
df_test[(df_test['matched_db_product'] == df_test['product']) &
        (df_test['matched_db_vendor'] == df_test['vendor']) &
        (df_test['true_version_in_predicted'] == 1)]

,cve_id,cpe_id_pk,vendor,product,version,descr,initial_cpe,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score,matched_db_product,matched_db_vendor,true_version_in_predicted
7,CVE-2023-31441,341885,ncia,advisor_network,2.2.0,"In NATO Communications and Information Agency anet (aka Advisor Network) through 3.3.0, an attacker can provide a crafted JSON file to sanitizeJson and cause an exception. This is related to the U...",cpe:2.3:a:ncia:advisor_network:2.2.0:*:*:*:*:*:*:*,"[[nato], [advisor network)], [through 3.3.0,]]","[[0.98350936], [0.7389296], [0.999821]]",[nato],[advisor network)],"[through 3.3.0,]",[0.98350936],[0.7389296],[0.999821],[nato],[0.98350936],[advisor network)],[0.7389296],advisor_network,ncia,1
12,CVE-2023-35094,353966,mpembed,wp_matterport_shortcode,1.7.1,Auth. (contributor+) Stored Cross-Site Scripting (XSS) vulnerability in Julien Berthelot / MPEmbed WP Matterport Shortcode plugin <= 2.1.4 versions.,cpe:2.3:a:mpembed:wp_matterport_shortcode:1.7.1:*:*:*:*:wordpress:*:*,"[[], [wp matterport shortcode plugin], [<= 2.1.4]]","[[], [0.8236863], [0.82940006]]",[],[wp matterport shortcode plugin],[<= 2.1.4],[],[0.8236863],[0.82940006],[],[],[wp matterport shortcode plugin],[0.8236863],wp_matterport_shortcode,mpembed,1
26,CVE-2015-8360,582917,atlassian,bamboo,2.5.5,An unspecified resource in Atlassian Bamboo before 5.9.9 and 5.10.x before 5.10.0 allows remote attackers to execute arbitrary Java code via serialized data to the JMS port.,cpe:2.3:a:atlassian:bamboo:2.5.5:*:*:*:*:*:*:*,"[[atlassian], [bamboo], [before 5.9.9, 5.10.x before 5.10.0]]","[[0.9997923], [0.99958223], [0.99993604, 0.9999495]]",[atlassian],[bamboo],"[before 5.9.9, 5.10.x before 5.10.0]",[0.9997923],[0.99958223],"[0.99993604, 0.9999495]",[atlassian],[0.9997923],[bamboo],[0.99958223],bamboo,atlassian,1
30,CVE-2021-43009,646632,opservices,opmon,9.9,A Cross Site Scripting (XSS) vulnerability exists in OpServices OpMon through 9.11 via the search parameter in the request URL.,cpe:2.3:a:opservices:opmon:9.9:*:*:*:*:*:*:*,"[[], [opmon], [through 9.11]]","[[], [0.9972307], [0.9998847]]",[],[opmon],[through 9.11],[],[0.9972307],[0.9998847],[],[],[opmon],[0.9972307],opmon,opservices,1
34,CVE-2022-24124,114598,casbin,casdoor,1.7.2,"The query API in Casdoor before 1.13.1 has a SQL injection vulnerability related to the field and value parameters, as demonstrated by api/get-organizations.",cpe:2.3:a:casbin:casdoor:1.7.2:*:*:*:*:*:*:*,"[[], [casdoor], [before 1.13.1]]","[[], [0.9969404], [0.9999317]]",[],[casdoor],[before 1.13.1],[],[0.9969404],[0.9999317],[],[],[casdoor],[0.9969404],casdoor,casbin,1
35,CVE-2023-1554,298377,fullworksplugins,quick_paypal_payments,5.7.4,"The Quick Paypal Payments WordPress plugin before 5.7.26.4 does not sanitise and escape some of its settings, which could allow high privilege users such as admin to perform Stored Cross-Site Scri...",cpe:2.3:a:fullworksplugins:quick_paypal_payments:5.7.4:*:*:*:*:wordpress:*:*,"[[], [quick paypal payments, wordpress], [before 5.7.26.4]]","[[], [0.9503791, 0.8185743], [0.999895]]",[],"[quick paypal payments, wordpress]",[before 5.7.26.4],[],"[0.9503791, 0.8185743]",[0.999895],[],[],[quick paypal payments],[0.9503791],quick_paypal_payments,fullworksplugins,1
37,CVE-2016-9132,443587,botan_project,botan,1.9.13,"In Botan 1.8.0 through 1.11.33, when decoding BER data an integer overflow could occur, which would cause an incorrect length field to be computed. Some API callers may use the returned (incorrect...",cpe:2.3:a:botan_project:botan:1.9.13:*:*:*:*:*:*:*,"[[], [botan], [1.8.0 through 1.11.33,]]","[[], [0.9931213], [0.9999437]]",[],[botan],"[1.8.0 through 1.11.33,]",[],[0.9931213],[0.9999437],[],[],[botan],[0.9931213],botan,botan_project,1
48,CVE-2023-24998,315104,apache,commons_fileupload,1.2.2,Apache Commons FileUpload before 1.5 does not limit the number of request pa